In [13]:
from datetime import timedelta
import cv2
import numpy as np
import os

In [14]:
#!pip install opencv-python

In [15]:
SAVING_FRAMES_PER_SECOND = 25

In [16]:
def format_timedelta(td):
    """
    Установим формат дат: исключим микросекунды и сохраним миллисекунды
    """
    
    result = str(td)
    
    try:
        result, ms = result.split(".")
        
    except ValueError:
        return result + ".00".replace(":","-")
    ms = int(ms)
    ms = round(ms/1e4)
    return f"{result}.{ms:02}".replace(":","-")

def get_saving_frames_duration(cap, saving_fps):
    """
    Возвращает список последовательностей, в которые следует сохранять кадры
    """
    s = []
    # получаем продолжительность клипа, 
    # разделив количество кадров на количество кадров в секунду
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)

    print('clip_duration: ', clip_duration)
    
    for i in np.arange(0, clip_duration, 1/saving_fps):
        s.append(round(i, 2))
    return s     
    
    

In [17]:
def main(video_file):
    filename, _ = os.path.splitext(video_file)
    filename += '-opencv'
    
    # создаем папку по названию файла
    if not os.path.isdir(filename):
        os.mkdir(filename)
        
    # читаем видео файл
    cap = cv2.VideoCapture(video_file)
    
    # получаем FPS видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    
    # получаем список длительностей для сохранения
    saving_frames_durations = get_saving_frames_duration(cap,
                                  saving_frames_per_second)
    
    # запускаем цикл
    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # выходим из цикла, если нет фреймов для чтения
            print('Фреймы для чтения закончились')
            break
            
        # получаем продолжительность, разделив количество кадров на FPS
        frame_duration = count / fps
        
        try:
            # получить самую раннюю продолжительность для сохранения
            closest_duration = saving_frames_durations[0]
        except IndexError:
            print('point 2')
            # список пуст, все кадрый длительности сохранены
            break
        if frame_duration >= closest_duration:
            # если ближайшая длительность меньше или равна
            # длительности кадра, затем сохраняем фрейм
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            
            cv2.imwrite(os.path.join(filename,
                                    f'frame{frame_duration_formatted}.jpg'),
                       frame)
            
            # удалить толчку продолжительности из списка, так как эта точка
            # длительности уже сохранена
            
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                print('point 3')
                
            # увеличить количество кадров
            count += 1

In [18]:
video_file = "Queen of Hearts in  Alice's Adventures in Wonderland.mp4"
main(video_file)
print('done')

clip_duration:  264.28
Фреймы для чтения закончились
done
